In [1]:
import tensorflow as tf
import input_data
mnist = input_data.read_data_sets("../data/MNIST_data/", one_hot=True)
checkpoint_dir = "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/seojey/ML lec/model/"

def xaver_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

learning_rate = 0.001
training_epochs = 20 # node count
batch_size = 100    # 
display_step = 1    # ?

# tf Graph Input
x = tf.placeholder("float", [None, 784])  # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10])  # 0-9 digits recognition => 10 classes
dropout_rate = tf.placeholder("float")

# Create model

# Set model weights
## W % b가 아래와 같이 나눠진 이유는?
## --> input & output 

W1 = tf.get_variable("W1", shape=[784, 500], initializer=xaver_init(784, 500))
W2 = tf.get_variable("W2", shape=[500, 256], initializer=xaver_init(500, 256))
W3 = tf.get_variable("W3", shape=[256, 10], initializer=xaver_init(256, 10))
W4 = tf.get_variable("W4", shape=[10, 10], initializer=xaver_init(10, 10))
W5 = tf.get_variable("W5", shape=[10, 10], initializer=xaver_init(10, 10))

## 왜 3개의 b가 필요한가?
b1 = tf.Variable(tf.zeros([500]))
b2 = tf.Variable(tf.zeros([256]))
b3 = tf.Variable(tf.zeros([10]))
b4 = tf.Variable(tf.zeros([10]))
b5 = tf.Variable(tf.zeros([10]))

# Construct model
## relu 와 softmax가 동일한 결과를 가져오는가?
_L1 = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))  # Softmax 대신 relu
L1 = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))  
L2 = tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))  
L3 = tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.add(tf.matmul(L3, W4), b4))  
L4 = tf.nn.dropout(_L4, dropout_rate)

activation = tf.add(tf.matmul(L4, W5), b5)  


# Minimize error using cross entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(activation, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Gradient Descent 대신 adom
# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver() # Saves and restores variables.

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print ('load learning')
        saver.restore(sess, ckpt.model_checkpoint_path)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, dropout_rate:1.0})
            # Compute average loss

        # Display logs per epoch step

        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, dropout_rate:1.0}) / total_batch
        if epoch % display_step == 0: # Softmax
            print ("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            #print (sess.run(b3))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels, dropout_rate:1.0}))

    saver.save(sess, checkpoint_dir + 'model.ckpt')
    
    

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/seojey/ML lec/ML lec 7 - Perf validation with Training-test set/input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
load learning


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [10] rhs shape= [256,10]
	 [[Node: save/Assign_6 = Assign[T=DT_FLOAT, _class=["loc:@Variable_2"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_2, save/restore_slice_6)]]
Caused by op 'save/Assign_6', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/